In [0]:
!git clone https://github.com/emanhamed/Houses-dataset

Cloning into 'Houses-dataset'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 2166 (delta 0), reused 0 (delta 0), pack-reused 2161
Receiving objects: 100% (2166/2166), 176.31 MiB | 42.61 MiB/s, done.
Resolving deltas: 100% (18/18), done.


In [0]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import glob
import cv2
import os

In [0]:
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model

In [0]:
def load_house_attributes(inputPath):
    cols = ["bedrooms", "bathrooms", "area", "zipcode", "price"]
    df = pd.read_csv(inputPath, sep=' ', header=None, names=cols)
    
    zipcodes = df['zipcode'].value_counts().keys().tolist()
    counts   = df['zipcode'].value_counts().tolist()
    #df.head()
    
    for (zipcode, count) in zip(zipcodes, counts):
      
        if count < 25:
            idxs = df[df['zipcode'] == zipcode].index
            df.drop(idxs, inplace=True)
            
    return df
  
def process_house_attributes(df, train, test):
    continuous = ['bedrooms', 'bathrooms', 'area']
    
    cs               = MinMaxScaler()
    train_continuous = cs.fit_transform(train[continuous])
    test_continuous  = cs.transform(test[continuous])
    
    zip_binarizer     = LabelBinarizer().fit(df['zipcode'])
    train_categorical = zip_binarizer.transform(train['zipcode'])
    test_categorical  = zip_binarizer.transform(test['zipcode'])
    
    train_X = np.hstack([train_categorical, train_continuous])
    test_X  = np.hstack([test_categorical, test_continuous])
    
    return (train_X, test_X)
  
def load_house_images(df, inputPath):
    images = []
    
    for i in df.index.values:
        base_path   = os.path.sep.join([inputPath, "{}_*".format(i + 1)])
        house_paths = sorted(list(glob.glob(base_path)))
        
        input_images = []
        output_image = np.zeros((64, 64, 3), dtype="uint8")
        
        for house_path in house_paths:
            image = cv2.imread(house_path)
            image = cv2.resize(image, (32, 32))
            input_images.append(image)
            
        output_image[0:32, 0:32]   = input_images[0]
        output_image[0:32, 32:64]  = input_images[1]
        output_image[32:64, 32:64] = input_images[2]
        output_image[32:64, 0:32]  = input_images[3]
        
        images.append(output_image)
        
    return np.array(images)
        

In [0]:
def create_cnn(width, height, depth, filters=(16, 32, 64), regress=False):
  
    input_shape = (height, width, depth)
    chan_dim = -1
    
    inputs = Input(shape=input_shape)
    
    for (i, f) in enumerate(filters):
        if i == 0:
            x = inputs
            
        x = Conv2D(f, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chan_dim)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        
    x = Flatten()(x)
    x = Dense(16)(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=chan_dim)(x)
    x = Dropout(0.5)(x)
    
    x = Dense(4)(x)
    x = Activation("relu")(x)
    
    if regress:
        x = Dense(1, activation="linear")(x)
        
    model = Model(inputs, x)
    
    return model

In [0]:
cwd = os.getcwd()
house_jpg_dir = os.path.join(cwd, 'Houses-dataset', 'Houses Dataset')
houses_info   = os.path.join(house_jpg_dir, 'HousesInfo.txt')

df = load_house_attributes(houses_info)

In [0]:
df.head()

,bedrooms,bathrooms,area,zipcode,price
30,5,3.0,2520,93446,789000
32,3,2.0,1802,93446,365000
39,3,3.0,2146,93446,455000
80,4,2.5,2464,91901,599000
81,2,2.0,1845,91901,529800


In [0]:
images = load_house_images(df, house_jpg_dir)
print(f"{len(images)} images.")

362 images.


In [0]:
# Scaling pixel intensities to the range [0, 1].
images = images / 255.0

In [0]:
split = train_test_split(df, images, test_size=0.25, random_state=42)
(trainAttrX, testAttrX, trainImagesX, testImagesX) = split

In [0]:
maxPrice = trainAttrX["price"].max()
trainY = trainAttrX["price"] / maxPrice
testY = testAttrX["price"] / maxPrice

In [0]:
model = create_cnn(64, 64, 3, regress=True)
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)
model.summary()

W0726 21:47:28.497643 139817191442304 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0726 21:47:28.823161 139817191442304 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0726 21:47:28.889328 139817191442304 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 16)        448       
_________________________________________________________________
activation_2 (Activation)    (None, 64, 64, 16)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 64, 64, 16)        64        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 32)        4640      
_________________________________________________________________
activation_3 (Activation)    (None, 32, 32, 32)        0         
__________

In [0]:
model.fit(trainImagesX, 
          trainY, 
          validation_data=(testImagesX, testY),	
          epochs=200, 
          batch_size=8)

model.save("housing_prices_images_model.h5")

Train on 271 samples, validate on 91 samples
Epoch 1/200
271/271 [==============================] - 4s 16ms/step - loss: 2757.7543 - val_loss: 5332.6830
Epoch 2/200
271/271 [==============================] - 3s 10ms/step - loss: 1852.7459 - val_loss: 1232.7159
Epoch 3/200
271/271 [==============================] - 3s 10ms/step - loss: 1226.1698 - val_loss: 488.2959
Epoch 4/200
271/271 [==============================] - 3s 9ms/step - loss: 1043.9225 - val_loss: 604.7963
Epoch 5/200
271/271 [==============================] - 2s 9ms/step - loss: 1023.1950 - val_loss: 485.1415
Epoch 6/200
271/271 [==============================] - 3s 9ms/step - loss: 897.8002 - val_loss: 741.5215
Epoch 7/200
271/271 [==============================] - 3s 10ms/step - loss: 609.2281 - val_loss: 361.3148
Epoch 8/200
271/271 [==============================] - 3s 10ms/step - loss: 526.9116 - val_loss: 243.1528
Epoch 9/200
271/271 [==============================] - 3s 10ms/step - loss: 456.6049 - val_loss: 208.51

In [0]:
preds = model.predict(testImagesX)

In [0]:
diff = preds.flatten() - testY
percentDiff = (diff / testY) * 100
absPercentDiff = np.abs(percentDiff)

In [0]:
mean = np.mean(absPercentDiff)
std = np.std(absPercentDiff)

In [0]:
print("avg. house price: ${:.2f}, std house price: ${:.2f}".format(
    df["price"].mean(), df["price"].std()))

print("mean: {:.2f}%, std: {:.2f}%".format(mean, std))

avg. house price: $533388.27, std house price: $493403.08
mean: 56.89%, std: 31.98%


In [0]:
from google.colab import files
files.download("housing_prices_images_model.h5")